In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.metrics import MeanAbsoluteError

wide_window = DataWindow(input_width=24, label_width=24, shift=1,label_columns=['traffic_volume'])



def objective(trial):
    # Hiperparámetros sugeridos por Optuna
    n_lstm_layers = trial.suggest_int("n_lstm_layers", 1, 3)
    n_units = trial.suggest_categorical("n_units", [32, 64, 128, 256])
    dropout_rate = trial.suggest_float("dropout", 0.0, 0.5)
    initial_lr = trial.suggest_float("initial_lr", 1e-4, 1e-2, log=True)
    final_lr = trial.suggest_float("final_lr", 1e-6, 1e-4, log=True)
    max_epochs = trial.suggest_int("max_epochs", 20, 50)
    patience = trial.suggest_int("patience", 3, 7)

    # Calcular pasos totales de entrenamiento
    num_batches_per_epoch = sum(1 for _ in wide_window.train)
    num_train_steps = num_batches_per_epoch * max_epochs

    # Scheduler
    lr_schedule = PolynomialDecay(
        initial_learning_rate=initial_lr,
        end_learning_rate=final_lr,
        decay_steps=num_train_steps
    )

    # Modelo LSTM
    model = Sequential()
    for i in range(n_lstm_layers):
        return_sequences = i < n_lstm_layers - 1
        if i == 0:
            model.add(LSTM(n_units, return_sequences=return_sequences))
        else:
            model.add(LSTM(n_units, return_sequences=return_sequences))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
    model.add(Dense(1))

    # Compilación con scheduler
    model.compile(
        loss='mse',
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=[MeanAbsoluteError()]
    )

    # Early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )

    # Entrenamiento
    history = model.fit(
        wide_window.train,
        validation_data=wide_window.val,
        epochs=max_epochs,
        callbacks=[early_stopping],
        verbose=0
    )

    return min(history.history['val_loss'])

# Ejecutar la optimización
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Mostrar mejores hiperparámetros
print("Mejores hiperparámetros encontrados:")
print(study.best_trial.params)

usar con la clase datawindow